In [1]:
import pandas as pd
import numpy as np
import math


url = "https://raw.githubusercontent.com/murpi/wilddata/master/quests/spotify.zip"
df_music = pd.read_csv(url)
df_music.shape

(232725, 18)

In [2]:
# There is two genre with similar name: "Children’s Music" and "Children's Music". I rename it before a merge with the following get dummies
df_music["genre"].replace('Children’s Music', "Children's Music", inplace=True)

# Get dummies on genre for Machine Learning Usage
df_music_dumies = pd.concat([df_music , df_music['genre'].str.get_dummies()], axis = 1)

# Factorization of "mode"
df_music_dumies['mode'] = df_music_dumies['mode'].factorize()[0]

# We remove song with a duration below 90 seconds et above 900 seconds
df_music_dumies = df_music_dumies.loc[df_music_dumies['duration_ms'] >= 90000]
df_music_dumies = df_music_dumies.loc[df_music_dumies['duration_ms'] <= 900000]

# We remove Soundtrack and Movie genre
df_music_dumies = df_music_dumies[(df_music_dumies['genre'] != 'Soundtrack') & (df_music_dumies['genre'] != 'Movie')]

In [3]:
# Function to split the popularity in 5 classes in order to use a classification Machine Learning method (suggested by Tarik as he says it's better than Linear Regression here)
def popularity_score(popularity):
  popularity = popularity / 20
  popularity = math.ceil(popularity)
  return popularity

In [4]:
# Applying the function
df_music_dumies["popularity_score"] = df_music["popularity"].apply(popularity_score)

In [6]:
# We remove useless columns
cols = ['genre', 'artist_name', 'track_id', 'key', 'time_signature']
df_music_dumies.drop(cols, axis=1, inplace=True)

In [7]:
# There are cells with a popularity of 0 and after many tests it seems we have better Machine Learning accuracy score when we remove it
df_music_without_0_pop = df_music_dumies[df_music_dumies['popularity_score'] != 0]

In [9]:
df_music_without_0_pop.to_csv('..\data\dataset_algo.csv.zip', header = True, compression='zip')